<a href="https://colab.research.google.com/github/ewotawa/secure_private_ai/blob/master/Section_4_Encrypted_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

In [1]:
# PySyft

!pip install syft

import syft as sy

# PyTorch

!pip install torch
!pip install torchvision

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import datasets, transforms

# Numpy

import numpy as np

# time

import time

     |████████████████████████████████| 256kB 2.6MB/s 
     |████████████████████████████████| 204kB 36.3MB/s 
     |████████████████████████████████| 1.4MB 39kB/s 
     |████████████████████████████████| 256kB 39.7MB/s 
     |████████████████████████████████| 81kB 14.4MB/s 
     |████████████████████████████████| 460kB 37.8MB/s 
     |████████████████████████████████| 389kB 39.2MB/s 
     |████████████████████████████████| 51kB 16.7MB/s 
     |████████████████████████████████| 266kB 45.2MB/s 
     |████████████████████████████████| 122kB 41.0MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067085 sha256=e068f9431787721b44563fbf9ff9cc1774ef77e1a45a53de5f71203122826bfa
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=34ad78578b768c914c5aea398579b873426d5a8b463a8fa374583baeee4150ed
  Stored in dire

W0808 02:07:18.436465 140370259122048 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0808 02:07:18.456842 140370259122048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [0]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [3]:
x = encrypt(encode(5.5))
x

[191399845254032188583941495419710394154,
 146112633277576147135582787737676387628,
 250434212418726158421366272403894707044]

In [4]:
y = encrypt(encode(2.3))
y

[62675146043691001422013069077952476457,
 200272344583655953542989332330522902702,
 31025854847820292105442876372120365253]

In [5]:
z = add(x,y)
z

(254074991297723190005954564497662870611,
 52411632386064853608126842287833545917,
 281460067266546450526809148776015072297)

In [6]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [0]:
field = 23740629843760239486723

In [0]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [9]:
(bob_x_share + alices_x_share) % field

5

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [11]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [12]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [13]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [0]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [15]:
(bob_z_share + alice_z_share) % field

4

In [0]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [18]:
bob_x_share + alice_x_share

15

In [19]:
bob_y_share + alice_y_share

11

In [20]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [0]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [22]:
x = encrypt(encode(5.5))
x

[135921453730543279725866724856646345035,
 130368345829642556347758822663336361297,
 27683545914981410996819730260933038081]

In [0]:
z = imul(x, 3)

In [24]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [0]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

In [26]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

W0808 02:07:30.117150 140370259122048 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0808 02:07:30.125979 140370259122048 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0808 02:07:30.127928 140370259122048 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [30]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [31]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [32]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [33]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [34]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [35]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [37]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [38]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [39]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [40]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [41]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [42]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [0]:
# try this project here!

###Instructor's work

Rationale for exercise:
* to show the general nature of encrypted computation - it can be based off of tensor computations. 

Process:
* take individual strings - key-value database of string representations
*  convert strongs to tensor operations.
* show how tesor operations can be used to perform queries.

Feature
* encrypted database: makes it so that the database owner 
 * can't see any of the data (the data is encrypted while it's in the database)
 * can't see what people are querying
 * can't see the results of the queries
* because all the values are encrypted using secure MPC (multi-party computation), you can have a group of database owners: multiple individuals that have joint ownership and joint governance over people's ability to query and use information. 


Come up with numerical representations for strings. 

Two general options: 
* one-hot representation
* integer representations

In [0]:
# choose what subset of characters we want to support on our database. 
# create lookup tables that map characters to integers
char2index = {}
index2char = {}

In [0]:
import string

# what do we want to encode
for i,char in enumerate(' ' + string.ascii_lowercase + '1234567890' + string.punctuation):
  char2index[char] = i
  index2char[i] = char

In [0]:
str_input = "Hello"
max_length = 8

In [0]:
# integer representation
def string2values(str_input, max_length):
  
  # trim any strings that are too long, convert them to lowercase
  str_input = str_input[:max_length].lower()

  # if a string is too short, pad it with .
  if(len(str_input) < max_length):
    str_input = str_input + "." * (max_length - len(str_input))

  values = list()

  for char in str_input:
    values.append(char2index[char])

  values_t = torch.tensor(values).long()
  
  return values_t

In [48]:
string2values("howdy!",8)

tensor([ 8, 15, 23,  4, 25, 37, 50, 50])

In [0]:
# one-hot encoding
def one_hot(index, length):
  # start with a tensor of zeros
  vect = torch.zeros(length).long()
  # whatever the index position is, set it to one
  vect[index] = 1
  return vect

In [50]:
one_hot(3,5)

tensor([0, 0, 0, 1, 0])

In [51]:
# representing characters
one_hot(char2index['p'], len(index2char))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
# integer representation
def string2one_hot_matrix(str_input, max_length):
  
  # trim any strings that are too long, convert them to lowercase
  str_input = str_input[:max_length].lower()

  # if a string is too short, pad it with .
  if(len(str_input) < max_length):
    str_input = str_input + "." * (max_length - len(str_input))

  char_vector = list()
  for char in str_input:
    char_v = one_hot(char2index[char], len(index2char)).unsqueeze(0)
    char_vector.append(char_v)
  
  result = torch.cat(char_vector, dim=0)
  return result

In [53]:
string2one_hot_matrix("Hello", 8)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

In [54]:
# 8 rows representing 8 characters, 69 columns representing the 69 possible characters to be encoded.
matrix = string2one_hot_matrix("Hello", 8)

matrix.shape

torch.Size([8, 69])

In [0]:
# database: store keys using one-hot encoding, store values using string-to-one-hot-matrix.

In [56]:
# test for characters in common
m_a = string2one_hot_matrix("abcdefg", 8)
m_b = string2one_hot_matrix("hijklmnop", 8)

# only positions where the same character is present in both strings will return a one. No matches returns a tensor of zero. 
(m_a * m_b).sum()

tensor(0)

In [57]:
# test for characters in common
m_a = string2one_hot_matrix("ThingOne", 8)
m_b = string2one_hot_matrix("ThingTwo", 8)

# only positions where the same character is present in both strings will return a one. No matches returns a tensor of zero. 
(m_a * m_b).sum()

tensor(5)

In [0]:
# Useful tool: allows us to use multiplication to tell whether or not a given key is a perfect match with a particular query.

In [59]:
# test for characters in common
m_a = string2one_hot_matrix("ThingOne", 8)
m_b = string2one_hot_matrix("ThingTwo", 8)

# sum along the 1 dimension: see how many characters are overlapping
(m_a * m_b).sum(1)

tensor([1, 1, 1, 1, 1, 0, 0, 0])

In [0]:
# Note: we mostly care whether or not the strings match completely. 

In [61]:
vect = (m_a * m_b).sum(1)
print(vect)

x = vect[0]

for i in range(vect.shape[0] - 1):
  x = x * vect[i + 1]
  
# Boolean value. 1: strings match, 2: strings do not match. 
# note: using only multiplication to encode a Boolean bit that determines whether or not the two keys match.
key_match = x
print(key_match)

tensor([1, 1, 1, 1, 1, 0, 0, 0])
tensor(0)


In [0]:
# create a dummy key-value pair

keys = list()
values = list()

keys.append(string2one_hot_matrix("key0", 8))
values.append(string2values("value0", 8))

keys.append(string2one_hot_matrix("key1", 8))
values.append(string2values("value1", 8))

In [0]:
def string_equal(str_a, str_b):
  vect = (str_a * str_b).sum(1)
  
  x = vect[0]

  for i in range(vect.shape[0] - 1):
    x = x * vect[i + 1]

  str_match = x
  return str_match

In [64]:
# a query to a key-value store first needs to compute whether the query matches any of the other keys.

query_str = "key1"

# convert to the correct representation

query_matrix = string2one_hot_matrix(query_str, 8)

# perform the query: iterate all keys and figure out whether any of them match

key_matches = list()

for key in keys:
  key_match = string_equal(key, query_matrix)
  key_matches.append(key_match)
  
print(key_matches)

[tensor(0), tensor(1)]


In [0]:
# we can mask out all the values that don't have matching keys.

In [66]:
print(values)

[tensor([22,  1, 12, 21,  5, 36, 50, 50]), tensor([22,  1, 12, 21,  5, 27, 50, 50])]


In [67]:
# result is the numerical representation of matching values
# adds up all the values but first masks out the ones that don't have matching keys.

result = values[0] * key_matches[0]

for i in range(len(values) - 1):
  result += values[i+1] * key_matches[i+1]
  
print(result)

tensor([22,  1, 12, 21,  5, 27, 50, 50])


In [0]:
# how do we get a tensor that is strings instead of numbers? 

def values2string(input_values):
  s = ""
  for value in input_values:
    s += index2char[int(value)]
  return s

In [69]:
values2string(result).replace(".", "")

'value1'

In [0]:
def query(query_str):
  query_matrix = string2one_hot_matrix(query_str, 8)

  key_matches = list()

  for key in keys:
    key_match = string_equal(key, query_matrix)
    key_matches.append(key_match)
    
  result = values[0] * key_matches[0]

  for i in range(len(values) - 1):
    result += values[i+1] * key_matches[i+1]
    
  return values2string(result).replace(".", "")

In [71]:
query("key0")

'value0'

In [0]:
# how do we pack this into a database?

In [0]:
# package the convenience functions

# integer representation
def string2values(str_input, max_length):
  
  # trim any strings that are too long, convert them to lowercase
  str_input = str_input[:max_length].lower()

  # if a string is too short, pad it with .
  if(len(str_input) < max_length):
    str_input = str_input + "." * (max_length - len(str_input))

  values = list()

  for char in str_input:
    values.append(char2index[char])

  values_t = torch.tensor(values).long()
  
  return values_t

# one-hot encoding
def one_hot(index, length):
  # start with a tensor of zeros
  vect = torch.zeros(length).long()
  # whatever the index position is, set it to one
  vect[index] = 1
  return vect

# integer representation
def string2one_hot_matrix(str_input, max_length):
  
  # trim any strings that are too long, convert them to lowercase
  str_input = str_input[:max_length].lower()

  # if a string is too short, pad it with .
  if(len(str_input) < max_length):
    str_input = str_input + "." * (max_length - len(str_input))

  char_vector = list()
  for char in str_input:
    char_v = one_hot(char2index[char], len(index2char)).unsqueeze(0)
    char_vector.append(char_v)
  
  result = torch.cat(char_vector, dim=0)
  return result

def string_equal(str_a, str_b):
  vect = (str_a * str_b).sum(1)
  
  x = vect[0]

  for i in range(vect.shape[0] - 1):
    x = x * vect[i + 1]

  str_match = x
  return str_match

def values2string(input_values):
  s = ""
  for value in input_values:
    s += index2char[int(value)]
  return s

In [0]:
class DB():
  
  def __init__(self, max_key_len=8, max_val_len=8):
    self.max_key_len = 8
    self.max_val_len = 8
    
    self.keys = list()
    self.values = list()

    self.keys.append(string2one_hot_matrix("key0", 8))
    self.values.append(string2values("value0", 8))

    self.keys.append(string2one_hot_matrix("key1", 8))
    self.values.append(string2values("value1", 8))
  
  def add_entry(self, key, value):
    self.keys.append(string2one_hot_matrix(key, self.max_key_len))
    self.values.append(string2values(value, self.max_val_len))
    
    
  def query(self, query_str):
    query_matrix = string2one_hot_matrix(query_str, 8)

    key_matches = list()

    for key in self.keys:
      key_match = string_equal(key, query_matrix)
      key_matches.append(key_match)

    result = self.values[0] * key_matches[0]

    for i in range(len(self.values) - 1):
      result += self.values[i+1] * key_matches[i+1]

    return values2string(result).replace(".", "")

In [0]:
db = DB()

In [151]:
db.query("key0")

'value0'

In [0]:
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")

In [153]:
db.query("key3")

'value3'

In [0]:
# now add encryption

class EncryptedDB():
  
  def __init__(self, *owners, max_key_len=8, max_val_len=8):
    self.max_key_len = 8
    self.max_val_len = 8
    
    self.keys = list()
    self.values = list()
    self.owners = owners
  
  def add_entry(self, key, value):
    key = string2one_hot_matrix(key, self.max_key_len)
    key = key.share(*self.owners)
    self.keys.append(key)
    
    value = string2one_hot_matrix(value, self.max_val_len)
    value = value.share(*self.owners)
    self.values.append(value)
        
  def query(self, query_str):
    query_matrix = string2one_hot_matrix(query_str, 8)
    
    query_matrix = query_matrix.share(*self.owners)

    key_matches = list()

    for key in self.keys:
      key_match = string_equal(key, query_matrix)
      key_matches.append(key_match)

    result = self.values[0] * key_matches[0]

    for i in range(len(self.values) - 1):
      result += self.values[i+1] * key_matches[i+1]
      
    result = result.get()

    return values2string(result).replace(".", "")

In [0]:
edb = EncryptedDB(bob, alice, secure_worker)

In [0]:
edb.add_entry("key0", "value0")
edb.add_entry("key1", "value1")

In [157]:
edb.query("key0")

TypeError: ignored

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [70]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(3.2824)
tensor(2.2900)
tensor(1.7782)
tensor(0.4646)
tensor(0.2601)
tensor(0.1478)
tensor(0.0836)
tensor(0.0483)
tensor(0.0298)
tensor(0.0203)
tensor(0.0151)
tensor(0.0121)
tensor(0.0101)
tensor(0.0086)
tensor(0.0075)
tensor(0.0065)
tensor(0.0057)
tensor(0.0051)
tensor(0.0045)
tensor(0.0040)


In [71]:
model(data)

tensor([[ 0.0205],
        [-0.0378],
        [ 0.9581],
        [ 1.0030]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [73]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:96788805685 -> alice:71262364326]
 	-> [PointerTensor | me:16502040509 -> bob:2593113142]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:48916567397 -> alice:77179339694]
 	-> [PointerTensor | me:69010484742 -> bob:81548539522]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:12874090904 -> alice:89626093199]
 	-> [PointerTensor | me:16880910293 -> bob:86340431345]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:90895915120 -> alice:33756292886]
 	-> [PointerTensor | me:4821978776 -> bob:63756350805]
 	*crypto provider: secure_worker*]

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [75]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:90836164363 -> alice:86998333138]
	-> [PointerTensor | me:88665994108 -> bob:34988800405]
	*crypto provider: secure_worker*

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [77]:
encrypted_prediction.get().float_precision()

tensor([[ 0.0200],
        [-0.0380],
        [ 0.9550],
        [ 1.0000]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [78]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 0s 0us/step


W0807 01:13:06.251907 139885005793152 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 22s 364us/sample - loss: 2.3022 - acc: 0.0938 - val_loss: 2.3010 - val_acc: 0.1002
Epoch 2/2
60000/60000 [==============================] - 21s 356us/sample - loss: 2.3005 - acc: 0.0959 - val_loss: 2.2992 - val_acc: 0.1019
Test loss: 2.299177756500244
Test accuracy: 0.1019


In [0]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [0]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [0]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [0]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [84]:
model.share(alice, bob, carol)

TypeError: ignored

## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [0]:
model.serve(num_requests=3)

## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [0]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org